<a href="http://agrum.org" target="blank"><img src="http://agrum.gitlab.io/theme/img/logoAgrum.png" align="left" style="height:100px"/></a><a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This pyAgrum's notebook is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

from pyAgrum.lib._utils.oslike import head

### generating data with missing values (at random)

In [2]:
src=gum.fastBN("A->B<-C->D->E<-B;D->F")
gum.generateCSV(src,"EM_nomissing.csv",5000,random_order=False)
src

(gum::BayesNet<double>@0x560f422cd470) BN{nodes: 6, arcs: 6, domainSize: 64, parameters: 28, compression ratio: 56% }

In [3]:
import pandas as pd
import numpy as np

def add_missing(src,dst,proba):
  df=pd.read_csv(src)
  mask=np.random.choice([True, False], size=df.shape,p=[proba,1-proba])
  df.mask(mask).to_csv(dst,na_rep='?',index=False,float_format='%.0f')

gum.generateCSV(src,"EM_nomissing.csv",5000,random_order=False)
add_missing("EM_nomissing.csv","EM_missing.csv",proba=0.1)

In [4]:
print("No missng")
head("EM_nomissing.csv")
print("Missing")
head("EM_missing.csv")

No missng
A,B,C,D,E,F
1,0,1,1,1,1
1,0,1,0,1,0
1,0,1,1,1,0
1,0,1,1,0,0
0,1,1,0,1,0
1,0,1,0,1,0
1,0,1,1,1,0
0,1,1,1,1,1
0,0,1,1,1,0

Missing
A,B,C,D,E,F
1,?,1,?,1,1
1,0,?,0,?,0
1,0,1,1,1,0
1,0,1,1,0,0
0,1,1,0,1,0
1,0,1,0,1,0
1,?,1,1,1,0
0,1,1,1,?,1
0,0,1,1,1,?



### learning with missing data

In [5]:
learner = gum.BNLearner('EM_missing.csv', ["?"])
print(f"Missing values in EM_missing.csv : {learner.hasMissingValues()}")

Missing values in EM_missing.csv : True


In [6]:
# this will fail : missing data !
# learner.learnParameters(src.dag())

In [7]:
learner.useEM(1e-3)
learner.useAprioriSmoothing()
bn=learner.learnParameters(src.dag())
print(f"# iterations : {learner.nbrIterations()}")
gnb.sideBySide(gnb.getInference(src),gnb.getInference(bn))

# iterations : 4


### learning with smaller error (and no smoothing)

In [8]:
learner = gum.BNLearner('EM_missing.csv', ["?"])
learner.setVerbosity(True)
learner.useEM(1e-8)
bn2=learner.learnParameters(src.dag())
print(f"# iterations : {learner.nbrIterations()}")
gnb.sideBySide(gnb.getInference(src),gnb.getInference(bn2))

# iterations : 13


In [9]:
print(learner.history())

(0.13764740612657408, 0.021727326687458724, 0.004381918309631967, 0.0009407567910606196, 0.00021824609738864458, 5.198470083400924e-05, 1.2739482836073836e-05, 3.2158608030732165e-06, 8.360392498523681e-07, 2.2346476202460753e-07, 6.123338591339003e-08, 1.713945955181076e-08, 4.881835208289358e-09)
